In [9]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from einops import rearrange
import math
from timm.models.layers import DropPath, to_2tuple, trunc_normal_
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, jaccard_score
from scipy.ndimage import distance_transform_edt
import pandas as pd
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable

class Luna16Dataset2D(Dataset):
    """
    Dataset class for LUNA16 NPY preprocessed data, handling 2D slices.
    """
    def __init__(self, base_dir, split='train', transform=None, target_size=(224, 224), debug=False):
        self.base_dir = base_dir
        self.split_dir = os.path.join(base_dir, split)
        self.images_dir = os.path.join(self.split_dir, 'images')
        self.labels_dir = os.path.join(self.split_dir, 'labels')
        self.transform = transform
        self.target_size = target_size
        self.debug = debug
        
        # Get all image files
        import glob
        self.image_files = sorted(glob.glob(os.path.join(self.images_dir, "*.npy")))
        
        # Get all label files
        self.label_files = sorted(glob.glob(os.path.join(self.labels_dir, "*.npy")))
        
        if debug:
            print(f"Found {len(self.image_files)} image files and {len(self.label_files)} label files")
        
        # Match images with labels based on extracted ID
        self.valid_files = []
        
        # Extract IDs from image filenames
        image_ids = {}
        for img_path in self.image_files:
            img_name = os.path.basename(img_path)
            # For patterns like "images_123.npy"
            if "_" in img_name:
                img_id = img_name.split("_")[1].split(".")[0]
                image_ids[img_id] = img_path
        
        # Extract IDs from label filenames and match with correct naming pattern
        for lbl_path in self.label_files:
            lbl_name = os.path.basename(lbl_path)
            
            # Check for different label naming patterns
            if lbl_name.startswith("masks_"):  # For "masks_123.npy" pattern
                lbl_id = lbl_name.split("_")[1].split(".")[0]
                if lbl_id in image_ids:
                    self.valid_files.append((image_ids[lbl_id], lbl_path))
            elif lbl_name.startswith("labels_"):  # For "labels_123.npy" pattern
                lbl_id = lbl_name.split("_")[1].split(".")[0]
                if lbl_id in image_ids:
                    self.valid_files.append((image_ids[lbl_id], lbl_path))
        
        if debug:
            print(f"Found {len(self.valid_files)} valid {split} samples with matching labels")
            if len(self.valid_files) > 0:
                print("First few matched pairs:")
                for i in range(min(3, len(self.valid_files))):
                    img, lbl = self.valid_files[i]
                    print(f"  {os.path.basename(img)} -> {os.path.basename(lbl)}")

    def __len__(self):
        return len(self.valid_files)

    def __getitem__(self, idx):
        img_path, label_path = self.valid_files[idx]
        
        try:
            # Load image and label
            image = np.load(img_path, allow_pickle=True)
            mask = np.load(label_path, allow_pickle=True)
            
            # Ensure data is in the right format
            if self.debug and idx == 0:
                print(f"Original shapes - Image: {image.shape}, Mask: {mask.shape}")
            
            # Handle potential object arrays
            if image.dtype == np.dtype('O') and image.size == 1:
                image = image.item()
            
            if mask.dtype == np.dtype('O') and mask.size == 1:
                mask = mask.item()
            
            # Ensure 2D format
            if image.ndim != 2:
                raise ValueError(f"Expected 2D image but got shape {image.shape}")
            
            if mask.ndim != 2:
                raise ValueError(f"Expected 2D mask but got shape {mask.shape}")
            
            # Normalize image (if needed)
            if np.max(image) > 1.0:
                image = image / np.max(image)
            
            # Ensure mask is binary
            mask = (mask > 0.5).astype(np.float32)
            
            # Convert to PyTorch tensors with channel dimension
            image_tensor = torch.tensor(image, dtype=torch.float32).unsqueeze(0)  # (1, H, W)
            mask_tensor = torch.tensor(mask, dtype=torch.float32).unsqueeze(0)    # (1, H, W)
            
            # Resize if needed (TransUNet typically expects 224x224)
            if image_tensor.shape[1:] != self.target_size:
                image_tensor = F.interpolate(
                    image_tensor.unsqueeze(0), 
                    size=self.target_size, 
                    mode='bilinear', 
                    align_corners=False
                ).squeeze(0)
                
                mask_tensor = F.interpolate(
                    mask_tensor.unsqueeze(0), 
                    size=self.target_size, 
                    mode='nearest'
                ).squeeze(0)
            
            # Create the data dictionary before transforms
            data = {"image": image_tensor, "label": mask_tensor}
            
            # Apply transforms
            if self.transform:
                try:
                    data = self.transform(data)
                except Exception as e:
                    print(f"Error applying transform {self.transform}")
                    import traceback
                    traceback.print_exc()
            
            return data
            
        except Exception as e:
            print(f"Error loading {img_path} or {label_path}: {e}")
            import traceback
            traceback.print_exc()
            
            # Return a zero tensor as fallback
            dummy_image = torch.zeros((1,) + self.target_size, dtype=torch.float32)
            dummy_mask = torch.zeros((1,) + self.target_size, dtype=torch.float32)
            return {"image": dummy_image, "label": dummy_mask}

# Transformer components

class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

class Attention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]

        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

# NOVEL MODIFICATION 1: Dual-Path Attention Module
class DualPathAttention(nn.Module):
    """
    Dual-Path Attention: Integrates spatial and channel attention pathways
    as proposed in recent medical segmentation research
    """
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0.):
        super().__init__()
        # Spatial attention path (standard multi-head attention)
        self.spatial_attn = Attention(
            dim=dim, 
            num_heads=num_heads,
            qkv_bias=qkv_bias,
            qk_scale=qk_scale,
            attn_drop=attn_drop,
            proj_drop=proj_drop
        )
        
        # Channel attention path
        self.channel_gate = nn.Sequential(
            nn.Linear(dim, dim // 8),
            nn.LayerNorm(dim // 8),
            nn.GELU(),
            nn.Linear(dim // 8, dim),
            nn.Sigmoid()
        )
        
        # Fusion module
        self.fusion = nn.Sequential(
            nn.Linear(dim * 2, dim),
            nn.LayerNorm(dim)
        )

    def forward(self, x):
        # Spatial attention pathway
        spatial_features = self.spatial_attn(x)
        
        # Channel attention pathway
        # Pool along sequence dimension
        channel_context = x.mean(dim=1, keepdim=True)
        channel_weights = self.channel_gate(channel_context)
        channel_features = x * channel_weights
        
        # Fusion of both pathways
        combined = torch.cat([spatial_features, channel_features], dim=-1)
        output = self.fusion(combined)
        
        return output

class Block(nn.Module):
    def __init__(
        self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., 
        attn_drop=0., drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm, 
        use_dual_attn=False  # Flag to use novel dual-path attention
    ):
        super().__init__()
        self.norm1 = norm_layer(dim)
        
        # Use either standard attention or dual-path attention
        if use_dual_attn:
            self.attn = DualPathAttention(
                dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale, 
                attn_drop=attn_drop, proj_drop=drop
            )
        else:
            self.attn = Attention(
                dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale, 
                attn_drop=attn_drop, proj_drop=drop
            )
            
        # NOVEL MODIFICATION 2: Stochastic Depth
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

    def forward(self, x):
        x = x + self.drop_path(self.attn(self.norm1(x)))
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x

class PatchEmbed(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_chans=1, embed_dim=768):
        super().__init__()
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)
        num_patches = (img_size[1] // patch_size[1]) * (img_size[0] // patch_size[0])
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = num_patches

        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        B, C, H, W = x.shape
        assert H == self.img_size[0] and W == self.img_size[1], \
            f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]})."
        x = self.proj(x).flatten(2).transpose(1, 2)
        return x

class TransformerEncoder(nn.Module):
    def __init__(
        self, img_size=224, patch_size=16, in_chans=1, num_classes=2, embed_dim=768, depth=12,
        num_heads=12, mlp_ratio=4., qkv_bias=True, qk_scale=None, representation_size=None, distilled=False,
        drop_rate=0., attn_drop_rate=0., drop_path_rate=0., norm_layer=nn.LayerNorm,
        use_dual_attn_layers=[3, 6, 9]  # Specify which layers use dual-path attention
    ):
        super().__init__()
        self.num_features = self.embed_dim = embed_dim
        self.num_tokens = 1
        
        self.patch_embed = PatchEmbed(
            img_size=img_size, patch_size=patch_size, in_chans=in_chans, embed_dim=embed_dim)
        num_patches = self.patch_embed.num_patches

        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + self.num_tokens, embed_dim))
        self.pos_drop = nn.Dropout(p=drop_rate)

        # NOVEL MODIFICATION 3: Progressive drop path rates
        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, depth)]
        
        self.blocks = nn.ModuleList([
            Block(
                dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[i], norm_layer=norm_layer,
                use_dual_attn=(i in use_dual_attn_layers)  # Use dual attention in specified layers
            )
            for i in range(depth)])
        
        self.norm = norm_layer(embed_dim)

        trunc_normal_(self.pos_embed, std=.02)
        trunc_normal_(self.cls_token, std=.02)
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def forward_features(self, x):
        B = x.shape[0]
        x = self.patch_embed(x)

        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.pos_embed
        x = self.pos_drop(x)

        features = []
        for i, blk in enumerate(self.blocks):
            x = blk(x)
            if i in [3, 6, 9]:  # Store features at different levels
                features.append(self.norm(x))
        
        return features

    def forward(self, x):
        return self.forward_features(x)

class ResNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResNetBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        
        out += self.shortcut(residual)
        out = self.relu(out)
        
        return out

class CNNEncoder(nn.Module):
    def __init__(self, in_channels=1):
        super(CNNEncoder, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(64, 64, 2, stride=1)
        self.layer2 = self._make_layer(64, 128, 2, stride=2)
        self.layer3 = self._make_layer(128, 256, 2, stride=2)
        self.layer4 = self._make_layer(256, 512, 2, stride=2)
        
    def _make_layer(self, in_channels, out_channels, blocks, stride):
        layers = []
        layers.append(ResNetBlock(in_channels, out_channels, stride))
        for _ in range(1, blocks):
            layers.append(ResNetBlock(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        # Store features at different levels for skip connections
        features = []
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        features.append(x)  # level 1 features
        
        x = self.maxpool(x)
        x = self.layer1(x)
        features.append(x)  # level 2 features
        
        x = self.layer2(x)
        features.append(x)  # level 3 features
        
        x = self.layer3(x)
        features.append(x)  # level 4 features
        
        x = self.layer4(x)
        features.append(x)  # level 5 features
        
        return features

# NOVEL MODIFICATION 4: Feature Refinement Module 
class FeatureRefinementModule(nn.Module):
    """
    Refines features by integrating both spatial and channel information
    from CNN and Transformer pathways.
    """
    def __init__(self, in_channels):
        super(FeatureRefinementModule, self).__init__()
        
        # Channel attention branch
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        
        self.channel_mlp = nn.Sequential(
            nn.Conv2d(in_channels, in_channels // 8, 1, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels // 8, in_channels, 1, bias=False)
        )
        
        # Spatial attention branch
        self.spatial_conv = nn.Conv2d(2, 1, kernel_size=7, padding=3, bias=False)
        
        # Final activation
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        # Original input
        residual = x
        
        # Channel attention
        avg_out = self.channel_mlp(self.avg_pool(x))
        max_out = self.channel_mlp(self.max_pool(x))
        channel_out = self.sigmoid(avg_out + max_out)
        
        # Apply channel attention
        x = x * channel_out
        
        # Spatial attention
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        spatial_in = torch.cat([avg_out, max_out], dim=1)
        spatial_out = self.sigmoid(self.spatial_conv(spatial_in))
        
        # Apply spatial attention
        x = x * spatial_out
        
        # Residual connection
        return x + residual

class DecoderBlock(nn.Module):
    def __init__(self, in_channels, skip_channels, out_channels, use_refinement=False):
        super(DecoderBlock, self).__init__()
        self.use_refinement = use_refinement
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels + skip_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
        
        # Add refinement module if specified
        if use_refinement:
            self.refine = FeatureRefinementModule(out_channels)
        
    def forward(self, x, skip=None):
        x = self.up(x)
        if skip is not None:
            x = torch.cat([x, skip], dim=1)
        x = self.conv(x)
        
        if self.use_refinement:
            x = self.refine(x)
            
        return x

# NOVEL MODIFICATION 5: Multi-Scale Skip Fusion
class MultiScaleSkipFusion(nn.Module):
    """
    Fuses skip connections from multiple scales using learnable weights
    """
    def __init__(self, channels):
        super(MultiScaleSkipFusion, self).__init__()
        self.weights = nn.Parameter(torch.ones(1, channels, 1, 1))
        self.gamma = nn.Parameter(torch.zeros(1))
        self.conv = nn.Conv2d(channels, channels, kernel_size=1)
        
    def forward(self, x, skip1, skip2=None):
        """
        x: main path features
        skip1, skip2: skip connection features (skip2 is optional)
        """
        # Adjust channel dimensions if needed
        if skip1.shape[1] != x.shape[1]:
            skip1 = self.conv(skip1)
        
        # Adjust spatial dimensions if needed
        if skip1.shape[2:] != x.shape[2:]:
            skip1 = F.interpolate(skip1, size=x.shape[2:], mode='bilinear', align_corners=True)
        
        # Apply weighted fusion
        if skip2 is not None:
            if skip2.shape[1] != x.shape[1]:
                skip2 = self.conv(skip2)
            
            if skip2.shape[2:] != x.shape[2:]:
                skip2 = F.interpolate(skip2, size=x.shape[2:], mode='bilinear', align_corners=True)
                
            weighted_skip = skip1 * self.weights + skip2 * (1 - self.weights)
        else:
            weighted_skip = skip1
            
        # Apply residual fusion with learnable scaling
        return x + self.gamma * weighted_skip

In [10]:
pip install torchviz graphviz onnx onnxruntime

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install torchsummary

Note: you may need to restart the kernel to use updated packages.


In [12]:
def visualize_model_architecture(model, output_dir):
    """
    Create a visualization of the model architecture
    
    Parameters:
    model: The TransUNet model
    output_dir: Directory to save the visualization
    """
    from torchviz import make_dot
    import torchvision
    
    # Create a dummy input
    dummy_input = torch.randn(1, 1, 224, 224).to(next(model.parameters()).device)
    
    # Forward pass to generate the graph
    output = model(dummy_input)
    
    # Create visualization
    dot = make_dot(output, params=dict(model.named_parameters()))
    
    # Configure visualization settings
    dot.attr('node', shape='box', style='filled', color='lightblue')
    dot.attr(rankdir='TB', size='12,12')  # Top to bottom layout
    
    # Save the visualization
    dot.render(os.path.join(output_dir, 'model_architecture'), format='png', cleanup=True)
    
    print(f"Model architecture visualization saved to {output_dir}")
    
    # Alternatively, you can also use ONNX for a more detailed visualization
    try:
        import onnx
        from onnx import helper as onnx_helper
        import onnxruntime as ort
        
        # Export model to ONNX
        torch.onnx.export(
            model,
            dummy_input,
            os.path.join(output_dir, 'model.onnx'),
            export_params=True,
            opset_version=11,
            do_constant_folding=True,
            input_names=['input'],
            output_names=['output'],
            dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
        )
        
        print(f"ONNX model exported to {output_dir}/model.onnx")
    except ImportError:
        print("ONNX export skipped - install onnx and onnxruntime for more detailed visualization")

In [13]:
class TransUNet(nn.Module):
    def __init__(
        self, 
        img_size=224, 
        in_channels=1, 
        num_classes=2,
        patch_size=16,
        embed_dim=768,
        depth=12,
        num_heads=12,
        mlp_ratio=4.,
        qkv_bias=True,
        drop_rate=0.1,
        attn_drop_rate=0.1,
        drop_path_rate=0.1,
        use_dual_attn=True,
        use_refinement=True,
        use_multiscale_fusion=True
    ):
        super(TransUNet, self).__init__()
        
        # Store the original image size for final upsampling
        self.img_size = img_size
        
        # Determine which layers use dual attention
        use_dual_attn_layers = [3, 6, 9] if use_dual_attn else []
        
        # Store configuration
        self.use_refinement = use_refinement
        self.use_multiscale_fusion = use_multiscale_fusion
        
        # CNN Encoder path
        self.cnn_encoder = CNNEncoder(in_channels=in_channels)
        
        # Transformer Encoder path
        self.transformer_encoder = TransformerEncoder(
            img_size=img_size,
            patch_size=patch_size,
            in_chans=in_channels,
            num_classes=num_classes,
            embed_dim=embed_dim,
            depth=depth,
            num_heads=num_heads,
            mlp_ratio=mlp_ratio,
            qkv_bias=qkv_bias,
            drop_rate=drop_rate,
            attn_drop_rate=attn_drop_rate,
            drop_path_rate=drop_path_rate,
            use_dual_attn_layers=use_dual_attn_layers
        )
        
        # Transformer features projection layers
        self.conv_t_1 = nn.Conv2d(embed_dim, 512, kernel_size=1)
        self.conv_t_2 = nn.Conv2d(embed_dim, 256, kernel_size=1)
        self.conv_t_3 = nn.Conv2d(embed_dim, 128, kernel_size=1)
        
        # Multi-scale skip fusions (if enabled)
        if use_multiscale_fusion:
            self.skip_fusion1 = MultiScaleSkipFusion(512)
            self.skip_fusion2 = MultiScaleSkipFusion(256)
            self.skip_fusion3 = MultiScaleSkipFusion(128)
            self.skip_fusion4 = MultiScaleSkipFusion(64)
        
        # Decoder path
        self.decoder4 = DecoderBlock(512, 256, 256, use_refinement=use_refinement)
        self.decoder3 = DecoderBlock(256, 128, 128, use_refinement=use_refinement)
        self.decoder2 = DecoderBlock(128, 64, 64, use_refinement=use_refinement)
        self.decoder1 = DecoderBlock(64, 0, 32, use_refinement=use_refinement)
        
        # Final layer
        self.final_conv = nn.Conv2d(32, num_classes, kernel_size=1)
        
    def reshape_transformer_output(self, x, target_shape):
        """
        Reshape transformer features to match spatial dimensions of CNN features
        """
        # Remove cls token
        x = x[:, 1:, :]
        
        # Calculate spatial dimensions
        h = w = int(math.sqrt(x.size(1)))
        
        # Reshape to 2D
        x = x.transpose(1, 2).view(-1, x.size(2), h, w)
        
        # Resize to target shape
        if x.shape[2:] != target_shape[2:]:
            x = F.interpolate(x, size=target_shape[2:], mode='bilinear', align_corners=True)
            
        return x
        
    def forward(self, x):
        # CNN encoder path
        cnn_features = self.cnn_encoder(x)
        
        # Transformer encoder path
        # Transformer encoder path
        transformer_features = self.transformer_encoder(x)
        
        # Reshape transformer features and project to the right dimensions
        t_feat_shape4 = cnn_features[4].shape
        t_feat_shape3 = cnn_features[3].shape
        t_feat_shape2 = cnn_features[2].shape
        
        t_feat4 = self.reshape_transformer_output(transformer_features[2], t_feat_shape4)
        t_feat3 = self.reshape_transformer_output(transformer_features[1], t_feat_shape3)
        t_feat2 = self.reshape_transformer_output(transformer_features[0], t_feat_shape2)
        
        t_feat4 = self.conv_t_1(t_feat4)
        t_feat3 = self.conv_t_2(t_feat3)
        t_feat2 = self.conv_t_3(t_feat2)
        
        # Fuse CNN features with transformer features
        if self.use_multiscale_fusion:
            feat4 = self.skip_fusion1(cnn_features[4], t_feat4)
            feat3 = self.skip_fusion2(cnn_features[3], t_feat3)
            feat2 = self.skip_fusion3(cnn_features[2], t_feat2)
            feat1 = self.skip_fusion4(cnn_features[1], cnn_features[0])
        else:
            feat4 = cnn_features[4] + t_feat4
            feat3 = cnn_features[3] + t_feat3
            feat2 = cnn_features[2] + t_feat2
            feat1 = cnn_features[1]
        
        # Decoder path with skip connections
        d4 = self.decoder4(feat4, feat3)  # 256
        d3 = self.decoder3(d4, feat2)     # 128
        d2 = self.decoder2(d3, feat1)     # 64
        d1 = self.decoder1(d2)            # 32
        
        # Final classification layer
        output = self.final_conv(d1)  # num_classes
        
        # Ensure output size matches input size
        if output.shape[2:] != (self.img_size, self.img_size):
            output = F.interpolate(output, size=(self.img_size, self.img_size), 
                                 mode='bilinear', align_corners=True)
        
        return output

# NOVEL MODIFICATION 6: Boundary-Aware Loss Function
class BoundaryAwareLoss(nn.Module):
    """
    Loss function that gives higher weight to boundary regions
    """
    def __init__(self, weight=None, boundary_weight=5.0):
        super(BoundaryAwareLoss, self).__init__()
        self.cross_entropy = CrossEntropyLoss(weight=weight, reduction='none')
        self.boundary_weight = boundary_weight
        self.sobel_x = torch.tensor([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=torch.float32).view(1, 1, 3, 3)
        self.sobel_y = torch.tensor([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=torch.float32).view(1, 1, 3, 3)
        
    def forward(self, inputs, targets):
        # Check for size mismatch and resize if needed
        if inputs.shape[2:] != targets.shape[1:]:
            # Resize the inputs to match the targets
            inputs = F.interpolate(inputs, size=targets.shape[1:], 
                                 mode='bilinear', align_corners=True)
            
        # Print shapes for debugging
        print(f"Loss function - inputs shape: {inputs.shape}, targets shape: {targets.shape}")
        
        # Standard cross entropy loss
        ce_loss = self.cross_entropy(inputs, targets)
        
        # Compute boundary regions (using Sobel operator for edge detection)
        if self.sobel_x.device != targets.device:
            self.sobel_x = self.sobel_x.to(targets.device)
            self.sobel_y = self.sobel_y.to(targets.device)
            
        # Convert targets to one-hot
        targets_one_hot = F.one_hot(targets, num_classes=inputs.size(1)).permute(0, 3, 1, 2).float()
        
        # Apply Sobel filters to detect edges
        padding = nn.ReplicationPad2d(1)
        targets_padded = padding(targets_one_hot)
        
        edge_x = F.conv2d(targets_padded, self.sobel_x.expand(targets_one_hot.size(1), -1, -1, -1), groups=targets_one_hot.size(1))
        edge_y = F.conv2d(targets_padded, self.sobel_y.expand(targets_one_hot.size(1), -1, -1, -1), groups=targets_one_hot.size(1))
        
        # Calculate edge magnitude
        edge = torch.sqrt(edge_x**2 + edge_y**2)
        boundary = (edge > 0.5).float().sum(dim=1)
        
        # Create boundary weight map
        weight_map = torch.ones_like(targets, dtype=torch.float32)
        weight_map[boundary > 0] = self.boundary_weight
        
        # Apply weights to the loss
        weighted_loss = (ce_loss * weight_map).mean()
        
        return weighted_loss

# ------------------ Evaluation Metrics and Visualization Functions ------------------

def dice_score(preds, targets):
    """
    Compute Dice score between prediction and target
    """
    smooth = 1e-6
    preds = (preds > 0.5).float()
    targets = (targets > 0.5).float()
    
    intersection = torch.sum(preds * targets)
    union = torch.sum(preds) + torch.sum(targets)
    
    dice = (2. * intersection + smooth) / (union + smooth)
    return dice.item()

def evaluate_segmentation(pred, gt, metric_list=None):
    """
    Evaluate segmentation prediction against ground truth using multiple metrics
    
    Parameters:
    pred: Binary prediction mask (numpy array)
    gt: Binary ground truth mask (numpy array)
    metric_list: List of metric names to calculate
    
    Returns:
    Dictionary with calculated metrics
    """
    if metric_list is None:
        metric_list = ["dice", "iou", "precision", "recall", "f1", "hausdorff"]
    
    results = {}
    
    # Ensure binary masks
    pred = (pred > 0.5).astype(np.uint8)
    gt = (gt > 0.5).astype(np.uint8)
    
    # Handle empty masks
    if np.sum(gt) == 0 and np.sum(pred) == 0:
        # Special case: both masks are empty
        results["dice"] = 1.0 if "dice" in metric_list else None
        results["iou"] = 1.0 if "iou" in metric_list else None
        results["precision"] = 1.0 if "precision" in metric_list else None
        results["recall"] = 1.0 if "recall" in metric_list else None
        results["f1"] = 1.0 if "f1" in metric_list else None 
        results["hausdorff"] = 0.0 if "hausdorff" in metric_list else None
        return results
    
    if np.sum(gt) == 0:
        # Ground truth is empty but prediction is not
        results["dice"] = 0.0 if "dice" in metric_list else None
        results["iou"] = 0.0 if "iou" in metric_list else None
        results["precision"] = 0.0 if "precision" in metric_list else None
        results["recall"] = 0.0 if "recall" in metric_list else None
        results["f1"] = 0.0 if "f1" in metric_list else None
        results["hausdorff"] = float('inf') if "hausdorff" in metric_list else None
        return results
    
    if np.sum(pred) == 0:
        # Prediction is empty but ground truth is not
        results["dice"] = 0.0 if "dice" in metric_list else None
        results["iou"] = 0.0 if "iou" in metric_list else None
        results["precision"] = 0.0 if "precision" in metric_list else None
        results["recall"] = 0.0 if "recall" in metric_list else None
        results["f1"] = 0.0 if "f1" in metric_list else None
        results["hausdorff"] = float('inf') if "hausdorff" in metric_list else None
        return results
    
    # Flatten masks for metric calculation
    gt_flat = gt.flatten()
    pred_flat = pred.flatten()
    
    # Dice coefficient
    if "dice" in metric_list:
        intersection = np.sum(gt_flat * pred_flat)
        union = np.sum(gt_flat) + np.sum(pred_flat)
        dice = (2. * intersection + 1e-6) / (union + 1e-6)
        results["dice"] = dice
    
    # IoU (Jaccard Index)
    if "iou" in metric_list:
        iou = jaccard_score(gt_flat, pred_flat, zero_division=0)
        results["iou"] = iou
    
    # Precision, Recall, F1
    if any(m in metric_list for m in ["precision", "recall", "f1"]):
        if "precision" in metric_list:
            precision = precision_score(gt_flat, pred_flat, zero_division=0)
            results["precision"] = precision
            
        if "recall" in metric_list:
            recall = recall_score(gt_flat, pred_flat, zero_division=0)
            results["recall"] = recall
            
        if "f1" in metric_list:
            f1 = f1_score(gt_flat, pred_flat, zero_division=0)
            results["f1"] = f1
    
    # Hausdorff distance
    if "hausdorff" in metric_list:
        try:
            # Compute distance transforms
            if np.sum(gt) > 0 and np.sum(pred) > 0:
                gt_distance = distance_transform_edt(1 - gt)
                pred_distance = distance_transform_edt(1 - pred)
                
                # Compute Hausdorff distance
                hausdorff_gt_to_pred = np.max(gt_distance * pred)
                hausdorff_pred_to_gt = np.max(pred_distance * gt)
                hausdorff = max(hausdorff_gt_to_pred, hausdorff_pred_to_gt)
                results["hausdorff"] = hausdorff
            else:
                results["hausdorff"] = float('inf')
        except Exception as e:
            print(f"Error calculating Hausdorff distance: {e}")
            results["hausdorff"] = float('inf')
    
    return results

def generate_confusion_matrix(pred, gt):
    """
    Generate confusion matrix for binary segmentation
    
    Parameters:
    pred: Binary prediction mask (numpy array)
    gt: Binary ground truth mask (numpy array)
    
    Returns:
    Dictionary with TP, FP, TN, FN values and derived metrics
    """
    # Ensure binary masks
    pred = (pred > 0.5).astype(np.uint8)
    gt = (gt > 0.5).astype(np.uint8)
    
    # Flatten masks
    gt_flat = gt.flatten()
    pred_flat = pred.flatten()
    
    # Compute confusion matrix
    tn, fp, fn, tp = confusion_matrix(gt_flat, pred_flat, labels=[0, 1]).ravel()
    
    # Calculate additional metrics
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    ppv = tp / (tp + fp) if (tp + fp) > 0 else 0  # Positive Predictive Value
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0  # Negative Predictive Value
    
    return {
        "TP": tp,
        "FP": fp,
        "TN": tn,
        "FN": fn,
        "sensitivity": sensitivity,
        "specificity": specificity,
        "accuracy": accuracy,
        "ppv": ppv,
        "npv": npv
    }

def visualize_results(model, dataset, num_samples=5, device='cuda'):
    """
    Visualize model predictions and ground truth for a few samples
    """
    # Create output directory for visualizations
    os.makedirs('visualization_results', exist_ok=True)
    
    # Set model to evaluation mode
    model.eval()
    
    # Create a dataloader with batch size 1 for easy indexing
    loader = DataLoader(dataset, batch_size=1, shuffle=True)
    
    # Take the first num_samples samples
    for i, batch in enumerate(loader):
        if i >= num_samples:
            break
        
        # Get data
        image = batch["image"].to(device)
        mask = batch["label"].to(device)
        
        # Make prediction
        with torch.no_grad():
            output = model(image)
            prediction = torch.argmax(output, dim=1).cpu().numpy()[0]
        
        # Convert to numpy arrays for visualization
        image_np = image.cpu().numpy()[0, 0]  # Get the first channel
        mask_np = mask.cpu().numpy()[0, 0]
        
        # Create figure with 3 subplots
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        # Plot original image
        axes[0].imshow(image_np, cmap='gray')
        axes[0].set_title('Original Image')
        axes[0].axis('off')
        
        # Plot ground truth mask
        axes[1].imshow(image_np, cmap='gray')
        mask_overlay = np.ma.masked_where(mask_np == 0, mask_np)
        axes[1].imshow(mask_overlay, cmap='jet', alpha=0.6)
        axes[1].set_title('Ground Truth')
        axes[1].axis('off')
        
        # Plot prediction
        axes[2].imshow(image_np, cmap='gray')
        pred_overlay = np.ma.masked_where(prediction == 0, prediction)
        axes[2].imshow(pred_overlay, cmap='plasma', alpha=0.6)
        axes[2].set_title('Prediction')
        axes[2].axis('off')
        
        # Save figure
        plt.tight_layout()
        plt.savefig(f'visualization_results/sample_{i}.png')
        plt.close()
        
    print(f"Saved {num_samples} visualization samples to 'visualization_results' directory")

def visualize_sample(image, mask, prediction, metrics, output_dir, sample_id):
    """
    Visualize original image, ground truth mask, and prediction with metrics
    
    Parameters:
    image: Original image (numpy array)
    mask: Ground truth mask (numpy array)
    prediction: Prediction mask (numpy array)
    metrics: Dictionary of evaluation metrics
    output_dir: Directory to save visualization
    sample_id: Sample identifier
    """
    # Create figure with 3 subplots
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Plot original image
    axes[0].imshow(image, cmap='gray')
    axes[0].set_title('Original Image')
    axes[0].axis('off')
    
    # Plot ground truth mask
    axes[1].imshow(image, cmap='gray')
    mask_overlay = np.ma.masked_where(mask == 0, mask)
    axes[1].imshow(mask_overlay, cmap='jet', alpha=0.6)
    axes[1].set_title('Ground Truth')
    axes[1].axis('off')
    
    # Plot prediction
    axes[2].imshow(image, cmap='gray')
    pred_overlay = np.ma.masked_where(prediction == 0, prediction)
    axes[2].imshow(pred_overlay, cmap='plasma', alpha=0.6)
    
    # Add metric values to title
    title = f'Prediction\nDice: {metrics["dice"]:.4f}, IoU: {metrics["iou"]:.4f}'
    axes[2].set_title(title)
    axes[2].axis('off')
    
    # Save figure
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"sample_{sample_id}.png"))
    plt.close()

def visualize_boundary_overlap(image, mask, prediction, output_dir, sample_id):
    """
    Visualize boundary overlap between prediction and ground truth
    
    Parameters:
    image: Original image (numpy array)
    mask: Ground truth mask (numpy array)
    prediction: Prediction mask (numpy array)
    output_dir: Directory to save visualization
    sample_id: Sample identifier
    """
    # Create figure
    fig, ax = plt.subplots(figsize=(10, 10))
    
    # Plot original image
    ax.imshow(image, cmap='gray')
    
    # Extract boundaries
    from skimage import measure
    
    # Ground truth boundary
    if np.sum(mask) > 0:
        gt_contours = measure.find_contours(mask, 0.5)
        for contour in gt_contours:
            ax.plot(contour[:, 1], contour[:, 0], linewidth=2, color='green')
    
    # Prediction boundary
    if np.sum(prediction) > 0:
        pred_contours = measure.find_contours(prediction, 0.5)
        for contour in pred_contours:
            ax.plot(contour[:, 1], contour[:, 0], linewidth=2, color='red')
    
    # Add legend
    gt_patch = mpatches.Patch(color='green', label='Ground Truth')
    pred_patch = mpatches.Patch(color='red', label='Prediction')
    ax.legend(handles=[gt_patch, pred_patch], loc='upper right')
    
    ax.set_title('Boundary Comparison')
    ax.axis('off')
    
    # Save figure
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"boundary_comparison_{sample_id}.png"))
    plt.close()

def visualize_error_map(image, mask, prediction, output_dir, sample_id):
    """
    Visualize error map (TP, FP, FN, TN)
    """
    # Error categories
    tp = (prediction == 1) & (mask == 1)  # True Positive
    fp = (prediction == 1) & (mask == 0)  # False Positive
    fn = (prediction == 0) & (mask == 1)  # False Negative
    tn = (prediction == 0) & (mask == 0)  # True Negative
    
    # Create custom colormap for error visualization
    colors = [(0, 0, 0.5),    # TN - dark blue
              (1, 0.6, 0),    # FP - orange
              (0.7, 0, 0),    # FN - dark red
              (0, 0.7, 0)]    # TP - green
    
    cmap_name = 'error_categories'
    cm = LinearSegmentedColormap.from_list(cmap_name, colors, N=4)
    
    # Create error map
    error_map = np.zeros_like(mask)
    error_map[tn] = 0
    error_map[fp] = 1
    error_map[fn] = 2
    error_map[tp] = 3
    
    # Create figure
    fig, ax = plt.subplots(figsize=(10, 10))
    
    # Show original image with error overlay
    ax.imshow(image, cmap='gray')
    im = ax.imshow(error_map, cmap=cm, alpha=0.7, vmin=0, vmax=3)
    
    # Add legend
    legend_elements = [
        Line2D([0], [0], marker='s', color='w', markerfacecolor=colors[3], markersize=15, label='True Positive'),
        Line2D([0], [0], marker='s', color='w', markerfacecolor=colors[1], markersize=15, label='False Positive'),
        Line2D([0], [0], marker='s', color='w', markerfacecolor=colors[2], markersize=15, label='False Negative'),
        Line2D([0], [0], marker='s', color='w', markerfacecolor=colors[0], markersize=15, label='True Negative')
    ]
    
    ax.legend(handles=legend_elements, loc='upper right')
    ax.set_title('Segmentation Error Map')
    ax.axis('off')
    
    # Save figure
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"error_map_{sample_id}.png"))
    plt.close()

def plot_metric_distributions(metrics_df, output_dir):
    """
    Generate and save visualizations of metric distributions
    
    Parameters:
    metrics_df: DataFrame with evaluation metrics
    output_dir: Directory to save visualizations
    """
    # Create a directory for metric plots
    plots_dir = os.path.join(output_dir, "metric_plots")
    os.makedirs(plots_dir, exist_ok=True)
    
    # Plot distributions for key metrics
    for metric in ["dice", "iou", "precision", "recall", "f1"]:
        plt.figure(figsize=(10, 6))
        sns.histplot(metrics_df[metric], kde=True)
        plt.title(f'Distribution of {metric.upper()} Scores')
        plt.xlabel(f'{metric.upper()} Score')
        plt.ylabel('Frequency')
        plt.savefig(os.path.join(plots_dir, f"{metric}_distribution.png"))
        plt.close()
    
    # Plot correlation matrix between metrics
    metric_columns = ["dice", "iou", "precision", "recall", "f1", 
                     "sensitivity", "specificity", "accuracy", "ppv", "npv"]
    corr = metrics_df[metric_columns].corr()
    
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm', cbar=True)
    plt.title('Correlation Matrix of Evaluation Metrics')
    plt.tight_layout()
    plt.savefig(os.path.join(plots_dir, "metric_correlations.png"))
    plt.close()
    
    # Create boxplot of all metrics
    plt.figure(figsize=(12, 8))
    metrics_df[metric_columns].boxplot()
    plt.title('Distribution of Evaluation Metrics')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(os.path.join(plots_dir, "metrics_boxplot.png"))
    plt.close()

def generate_performance_report(metrics_df, output_dir):
    """
    Generate and save a comprehensive performance report
    
    Parameters:
    metrics_df: DataFrame with evaluation metrics
    output_dir: Directory to save report
    """
    report_path = os.path.join(output_dir, "performance_report.html")
    
    # Calculate average metrics
    avg_metrics = metrics_df.drop(columns=["sample_id", "TP", "FP", "TN", "FN"]).mean()
    
    # Calculate overall confusion matrix values
    total_tp = metrics_df["TP"].sum()
    total_fp = metrics_df["FP"].sum()
    total_tn = metrics_df["TN"].sum()
    total_fn = metrics_df["FN"].sum()
    
    # Generate HTML report
    html_content = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>Lung Nodule Segmentation Performance Report</title>
        <style>
            body {{ font-family: Arial, sans-serif; margin: 20px; }}
            h1, h2 {{ color: #2c3e50; }}
            table {{ border-collapse: collapse; width: 100%; margin-bottom: 20px; }}
            th, td {{ padding: 12px; text-align: left; border-bottom: 1px solid #ddd; }}
            th {{ background-color: #f2f2f2; }}
            .metric-value {{ font-weight: bold; color: #3498db; }}
            .confusion-matrix {{ display: flex; flex-wrap: wrap; margin-bottom: 20px; }}
            .matrix-cell {{ width: 150px; height: 100px; display: flex; align-items: center; 
                          justify-content: center; margin: 5px; color: white; font-weight: bold; }}
            .tp {{ background-color: #2ecc71; }} /* True Positive */
            .fp {{ background-color: #e74c3c; }} /* False Positive */
            .fn {{ background-color: #f39c12; }} /* False Negative */
            .tn {{ background-color: #3498db; }} /* True Negative */
        </style>
    </head>
    <body>
        <h1>Lung Nodule Segmentation Performance Report</h1>
        
        <h2>Average Performance Metrics</h2>
        <table>
            <tr>
                <th>Metric</th>
                <th>Value</th>
                <th>Description</th>
            </tr>
            <tr>
                <td>Dice Coefficient</td>
                <td class="metric-value">{avg_metrics["dice"]:.4f}</td>
                <td>Measure of overlap between prediction and ground truth (higher is better)</td>
            </tr>
            <tr>
                <td>IoU (Jaccard Index)</td>
                <td class="metric-value">{avg_metrics["iou"]:.4f}</td>
                <td>Ratio of intersection to union of prediction and ground truth (higher is better)</td>
            </tr>
            <tr>
                <td>F1 Score</td>
                <td class="metric-value">{avg_metrics["f1"]:.4f}</td>
                <td>Harmonic mean of precision and recall (higher is better)</td>
            </tr>
            <tr>
                <td>Precision</td>
                <td class="metric-value">{avg_metrics["precision"]:.4f}</td>
                <td>Ratio of true positives to all positive predictions (higher is better)</td>
            </tr>
            <tr>
                <td>Recall (Sensitivity)</td>
                <td class="metric-value">{avg_metrics["recall"]:.4f}</td>
                <td>Ratio of true positives to all actual positives (higher is better)</td>
            </tr>
            <tr>
                <td>Specificity</td>
                <td class="metric-value">{avg_metrics["specificity"]:.4f}</td>
                <td>Ratio of true negatives to all actual negatives (higher is better)</td>
            </tr>
            <tr>
                <td>Accuracy</td>
                <td class="metric-value">{avg_metrics["accuracy"]:.4f}</td>
                <td>Ratio of correct predictions to total predictions (higher is better)</td>
            </tr>
            <tr>
                <td>Hausdorff Distance</td>
                <td class="metric-value">{avg_metrics["hausdorff"]:.4f}</td>
                <td>Maximum distance between prediction and ground truth boundaries (lower is better)</td>
            </tr>
        </table>
        
        <h2>Overall Confusion Matrix</h2>
        
        <div class="confusion-matrix">
            <div class="matrix-cell tp">True Positive<br>{total_tp}</div>
            <div class="matrix-cell fp">False Positive<br>{total_fp}</div>
            <div class="matrix-cell fn">False Negative<br>{total_fn}</div>
            <div class="matrix-cell tn">True Negative<br>{total_tn}</div>
        </div>
        
        <h2>Additional Performance Indicators</h2>
        <table>
            <tr>
                <th>Metric</th>
                <th>Value</th>
                <th>Description</th>
            </tr>
            <tr>
                <td>Positive Predictive Value (PPV)</td>
                <td class="metric-value">{avg_metrics["ppv"]:.4f}</td>
                <td>Probability that a positive prediction is correct</td>
            </tr>
            <tr>
                <td>Negative Predictive Value (NPV)</td>
                <td class="metric-value">{avg_metrics["npv"]:.4f}</td>
                <td>Probability that a negative prediction is correct</td>
            </tr>
        </table>
        
        <p>
            This report summarizes the performance of the TransUNet model on the lung nodule segmentation task.
            The metrics indicate how well the model identifies and segments lung nodules in CT images.
        </p>
    </body>
    </html>
    """
    
    # Write the HTML report to file
    with open(report_path, 'w') as f:
        f.write(html_content)
    
    print(f"Performance report saved to {report_path}")
    return report_path

def evaluate_model(model, data_loader, device, output_dir="evaluation_results"):
    """
    Evaluate model using multiple metrics and generate visualizations
    
    Parameters:
    model: Trained model
    data_loader: DataLoader with samples to evaluate
    device: Device to run evaluation on
    output_dir: Directory to save evaluation results
    
    Returns:
    DataFrame with evaluation metrics for each sample
    """
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(os.path.join(output_dir, "sample_visualizations"), exist_ok=True)
    
    # Set model to evaluation mode
    model.eval()
    
    # Metrics to track
    all_metrics = []
    
    # Process each batch
    for batch_idx, batch in enumerate(tqdm(data_loader, desc="Evaluating")):
        # Get data
        images = batch["image"].to(device)
        masks = batch["label"].to(device)
        
        # Generate predictions
        with torch.no_grad():
            outputs = model(images)

        predictions = torch.argmax(outputs, dim=1)
        
        # Process each sample in batch
        for i in range(images.shape[0]):
            image = images[i, 0].cpu().numpy()
            mask = masks[i, 0].cpu().numpy()
            pred = predictions[i].cpu().numpy()
            
            # Calculate metrics
            metrics = evaluate_segmentation(
                pred, mask, 
                metric_list=["dice", "iou", "precision", "recall", "f1", "hausdorff"]
            )
            
            # Calculate confusion matrix and derived metrics
            confusion_metrics = generate_confusion_matrix(pred, mask)
            
            # Combine metrics
            all_metrics_sample = {**metrics, **confusion_metrics, "sample_id": batch_idx * data_loader.batch_size + i}
            all_metrics.append(all_metrics_sample)
            
            # Visualize and save every 10th sample
            if (batch_idx * data_loader.batch_size + i) % 10 == 0:
                sample_id = batch_idx * data_loader.batch_size + i
                output_subdir = os.path.join(output_dir, "sample_visualizations")
                
                # Basic visualization
                visualize_sample(image, mask, pred, metrics, output_subdir, sample_id)
                
                # Additional visualizations
                visualize_boundary_overlap(image, mask, pred, output_subdir, sample_id)
                visualize_error_map(image, mask, pred, output_subdir, sample_id)
    
    # Create DataFrame with all metrics
    metrics_df = pd.DataFrame(all_metrics)
    
    # Calculate and print average metrics
    avg_metrics = metrics_df.drop(columns=["sample_id", "TP", "FP", "TN", "FN"] if all(c in metrics_df.columns for c in ["TP", "FP", "TN", "FN"]) else ["sample_id"]).mean()
    print("\nAverage Metrics:")
    for metric, value in avg_metrics.items():
        print(f"{metric}: {value:.4f}")
    
    # Save metrics to CSV
    metrics_df.to_csv(os.path.join(output_dir, "detailed_metrics.csv"), index=False)
    avg_metrics.to_frame().T.to_csv(os.path.join(output_dir, "average_metrics.csv"), index=False)
    
    # Generate and save metric visualizations
    plot_metric_distributions(metrics_df, output_dir)
    
    # Generate performance report
    generate_performance_report(metrics_df, output_dir)
    
    return metrics_df

def visualize_attention_maps(model, image_tensor, device):
    """
    Visualize the attention maps from the Transformer part of TransUNet
    
    Parameters:
    model: Trained TransUNet model
    image_tensor: Input image tensor
    device: Device to run inference on
    
    Returns:
    Figure with attention visualizations
    """
    # Ensure model is in eval mode
    model.eval()
    
    # Forward pass through the model, but we need to capture intermediate attention maps
    # This is a simplified version - full implementation would require modifying the model
    # to return attention weights
    
    # For demonstration, we'll just create a mock visualization
    # In a real implementation, you would extract attention weights from the model
    
    # Get image as numpy array for visualization
    image_np = image_tensor.cpu().numpy()[0, 0]
    
    # Create figure for visualization
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    # Plot original image
    axes[0].imshow(image_np, cmap='gray')
    axes[0].set_title('Original Image')
    axes[0].axis('off')
    
    # Plot simulated attention maps (in a real implementation, these would be actual attention maps)
    import scipy.ndimage as ndimage
    
    # Create simulated attention maps by applying different filters to the image
    attention_maps = []
    # Simulate attention to edges
    attention_maps.append(ndimage.sobel(image_np))
    # Simulate attention to bright regions
    attention_maps.append(ndimage.gaussian_filter(image_np, sigma=2))
    # Simulate attention to texture
    attention_maps.append(ndimage.gaussian_laplace(image_np, sigma=2))
    # Simulate attention to corners
    attention_maps.append(ndimage.gaussian_gradient_magnitude(image_np, sigma=2))
    
    # Plot attention maps
    for i, attn_map in enumerate(attention_maps):
        im = axes[i+1].imshow(attn_map, cmap='inferno')
        axes[i+1].set_title(f'Attention Map {i+1}')
        axes[i+1].axis('off')
        
        # Add colorbar
        divider = make_axes_locatable(axes[i+1])
        cax = divider.append_axes("right", size="5%", pad=0.05)
        plt.colorbar(im, cax=cax)
    
    # Disable empty subplot if any
    if len(attention_maps) < 5:
        axes[5].axis('off')
    
    plt.tight_layout()
    return fig

def visualize_feature_maps(model, image_tensor, device, output_dir, sample_id):
    """
    Visualize feature maps from different layers of the model
    """
    # Ensure model is in eval mode
    model.eval()
    
    # Get image as numpy array
    image_np = image_tensor.cpu().numpy()[0, 0]
    
    # Create figure for visualization
    fig = plt.figure(figsize=(15, 10))
    gs = gridspec.GridSpec(3, 4)  # 3 rows, 4 columns
    
    # Plot original image
    ax = plt.subplot(gs[0, :2])
    ax.imshow(image_np, cmap='gray')
    ax.set_title('Original Image')
    ax.axis('off')
    
    # Create various filtered versions to simulate feature maps
    feature_maps = []
    import scipy.ndimage as ndimage
    
    # Edge detection
    feature_maps.append(('Edges', ndimage.sobel(image_np)))
    
    # Smoothing at different scales
    feature_maps.append(('Smoothed (σ=1)', ndimage.gaussian_filter(image_np, sigma=1)))
    feature_maps.append(('Smoothed (σ=3)', ndimage.gaussian_filter(image_np, sigma=3)))
    
    # Laplacian (detects edges and texture)
    feature_maps.append(('Laplacian', ndimage.laplace(image_np)))
    
    # Gradient magnitude
    feature_maps.append(('Gradient Magnitude', ndimage.gaussian_gradient_magnitude(image_np, sigma=1)))
    
    # Median filter (removes noise)
    feature_maps.append(('Median Filtered', ndimage.median_filter(image_np, size=3)))
    
    # Sharpening
    blurred = ndimage.gaussian_filter(image_np, 1)
    filter_blurred = ndimage.gaussian_filter(blurred, 1)
    sharpened = blurred + 1.5 * (blurred - filter_blurred)
    feature_maps.append(('Sharpened', sharpened))
    
    # Plot feature maps - FIX: Better position management to avoid out-of-bounds
    positions = [
        (0, 2), (0, 3),  # Top row, positions 2 and 3
        (1, 0), (1, 1), (1, 2), (1, 3),  # Middle row
        (2, 0), (2, 1), (2, 2), (2, 3)   # Bottom row
    ]
    
    # Plot feature maps
    for i, (title, feat_map) in enumerate(feature_maps):
        if i < len(positions):  # Make sure we don't exceed available positions
            pos = positions[i]
            ax = plt.subplot(gs[pos[0], pos[1]])
            im = ax.imshow(feat_map, cmap='viridis')
            ax.set_title(title)
            ax.axis('off')
            
            # Add colorbar
            divider = make_axes_locatable(ax)
            cax = divider.append_axes("right", size="5%", pad=0.05)
            plt.colorbar(im, cax=cax)
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"feature_maps_{sample_id}.png"))
    plt.close()

def train_model(model, train_loader, val_loader, num_epochs=50, device='cuda', output_dir='model_output'):
    """
    Train the model and track performance throughout training
    
    Parameters:
    model: Model to train
    train_loader: DataLoader for training data
    val_loader: DataLoader for validation data
    num_epochs: Number of epochs to train for
    device: Device to train on
    output_dir: Directory to save model checkpoints and training plots
    
    Returns:
    Trained model and training history
    """
    os.makedirs(output_dir, exist_ok=True)
    
    # Define loss function
    criterion = BoundaryAwareLoss()
    
    # Optimizer with AdamW and cosine learning rate schedule
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=1e-6)
    
    # Training history
    history = {
        'train_loss': [],
        'val_loss': [],
        'val_dice': [],
        'val_iou': [],
        'lr': []
    }
    
    best_val_dice = 0.0
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        train_loss = 0
        
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
            inputs = batch["image"].to(device)
            labels = batch["label"].squeeze(1).long().to(device)  # Convert to long for CrossEntropyLoss
            
            # Forward pass
            optimizer.zero_grad()
            outputs = model(inputs)
            
            # Compute loss
            loss = criterion(outputs, labels)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        # Update learning rate
        scheduler.step()
        current_lr = scheduler.get_last_lr()[0]
        
        # Calculate average training loss
        avg_train_loss = train_loss / len(train_loader)
        
        # Validation
        model.eval()
        val_loss = 0
        dice_scores = []
        iou_scores = []
        
        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
                inputs = batch["image"].to(device)
                labels = batch["label"].squeeze(1).long().to(device)
                
                # Forward pass
                outputs = model(inputs)
                
                # Compute loss
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
                # Compute dice score and IoU
                preds = torch.argmax(outputs, dim=1)
                # Convert to numpy for metric calculation
                preds_np = preds.cpu().numpy()
                labels_np = labels.cpu().numpy()
                
                # Calculate metrics for each sample in batch
                for i in range(preds_np.shape[0]):
                    metrics = evaluate_segmentation(
                        preds_np[i], labels_np[i], 
                        metric_list=["dice", "iou"]
                    )
                    dice_scores.append(metrics["dice"])
                    iou_scores.append(metrics["iou"])
        
        # Calculate average validation metrics
        avg_val_loss = val_loss / len(val_loader)
        avg_dice = np.mean(dice_scores)
        avg_iou = np.mean(iou_scores)
        
        # Update history
        history['train_loss'].append(avg_train_loss)
        history['val_loss'].append(avg_val_loss)
        history['val_dice'].append(avg_dice)
        history['val_iou'].append(avg_iou)
        history['lr'].append(current_lr)
        
        # Print statistics
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"  Train Loss: {avg_train_loss:.4f}")
        print(f"  Val Loss: {avg_val_loss:.4f}")
        print(f"  Dice Score: {avg_dice:.4f}")
        print(f"  IoU: {avg_iou:.4f}")
        print(f"  Learning Rate: {current_lr:.8f}")
        
        # Save best model
        if avg_dice > best_val_dice:
            best_val_dice = avg_dice
            torch.save(model.state_dict(), os.path.join(output_dir, "best_model.pth"))
            print(f"✓ New best model saved with Dice score: {avg_dice:.4f}")
            
            # Visualize results at best checkpoint
            if len(val_loader) > 0:
                batch = next(iter(val_loader))
                inputs = batch["image"].to(device)
                labels = batch["label"].to(device)
                
                with torch.no_grad():
                    outputs = model(inputs)
                    preds = torch.argmax(outputs, dim=1)
                
                # Visualize first sample in batch
                image = inputs[0, 0].cpu().numpy()
                mask = labels[0, 0].cpu().numpy()
                pred = preds[0].cpu().numpy()
                
                metrics = evaluate_segmentation(pred, mask, metric_list=["dice", "iou"])
                visualize_sample(image, mask, pred, metrics, output_dir, epoch)
    
    # Plot training history
    plot_training_history(history, output_dir)
    
    # Final model - either load best or use the last one
    model.load_state_dict(torch.load(os.path.join(output_dir, "best_model.pth")))
    
    return model, history

def plot_training_history(history, output_dir):
    """
    Plot training history metrics
    
    Parameters:
    history: Dictionary containing training metrics
    output_dir: Directory to save plots
    """
    plt.figure(figsize=(16, 12))
    
    # Plot training and validation loss
    plt.subplot(2, 2, 1)
    plt.plot(history['train_loss'], label='Training Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title('Loss During Training')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    # Plot validation Dice score
    plt.subplot(2, 2, 2)
    plt.plot(history['val_dice'], label='Validation Dice', color='green')
    plt.title('Dice Score During Training')
    plt.xlabel('Epoch')
    plt.ylabel('Dice Score')
    plt.legend()
    plt.grid(True)
    
    # Plot validation IoU
    plt.subplot(2, 2, 3)
    plt.plot(history['val_iou'], label='Validation IoU', color='purple')
    plt.title('IoU During Training')
    plt.xlabel('Epoch')
    plt.ylabel('IoU')
    plt.legend()
    plt.grid(True)
    
    # Plot learning rate
    plt.subplot(2, 2, 4)
    plt.plot(history['lr'], label='Learning Rate', color='orange')
    plt.title('Learning Rate Schedule')
    plt.xlabel('Epoch')
    plt.ylabel('Learning Rate')
    plt.legend()
    plt.grid(True)
    plt.yscale('log')
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'training_history.png'), dpi=300)
    plt.close()

def visualize_nodule_size_analysis(model, dataset, device, output_dir):
    """
    Analyze model performance by nodule size
    
    Parameters:
    model: Trained model
    dataset: Dataset to analyze
    device: Device to run inference on
    output_dir: Directory to save visualizations
    """
    model.eval()
    
    # Create directory for analysis
    analysis_dir = os.path.join(output_dir, "nodule_size_analysis")
    os.makedirs(analysis_dir, exist_ok=True)
    
    # Create dataloader with batch size 1
    loader = DataLoader(dataset, batch_size=1, shuffle=False)
    
    # Collect results by nodule size
    size_bins = [(0, 50), (50, 200), (200, 500), (500, 1000), (1000, float('inf'))]
    size_labels = ['Very Small', 'Small', 'Medium', 'Large', 'Very Large']
    results_by_size = {label: {'dice': [], 'iou': [], 'hausdorff': []} for label in size_labels}
    
    for batch in tqdm(loader, desc="Analyzing nodule sizes"):
        image = batch["image"].to(device)
        mask = batch["label"].to(device)
        
        # Forward pass
        with torch.no_grad():
            output = model(image)
            pred = torch.argmax(output, dim=1)
        
        # Convert to numpy
        mask_np = mask.cpu().numpy()[0, 0]
        pred_np = pred.cpu().numpy()[0]
        
        # Calculate nodule size (pixel count)
        nodule_size = np.sum(mask_np > 0.5)
        
        # Calculate metrics
        metrics = evaluate_segmentation(
            pred_np, mask_np, 
            metric_list=["dice", "iou", "hausdorff"]
        )
        
        # Add to appropriate size bin
        for (min_size, max_size), label in zip(size_bins, size_labels):
            if min_size <= nodule_size < max_size:
                results_by_size[label]['dice'].append(metrics['dice'])
                results_by_size[label]['iou'].append(metrics['iou'])
                results_by_size[label]['hausdorff'].append(metrics['hausdorff'])
                break
    
    # Create visualization of performance by nodule size
    metrics_to_plot = ['dice', 'iou', 'hausdorff']
    metric_titles = ['Dice Score', 'IoU', 'Hausdorff Distance']
    
    plt.figure(figsize=(18, 6))
    
    for i, (metric, title) in enumerate(zip(metrics_to_plot, metric_titles)):
        plt.subplot(1, 3, i+1)
        
        # Collect means and standard deviations
        means = []
        stds = []
        counts = []
        
        for label in size_labels:
            values = results_by_size[label][metric]
            if values:  # Only add if we have values
                means.append(np.mean(values))
                stds.append(np.std(values))
                counts.append(len(values))
            else:
                means.append(0)
                stds.append(0)
                counts.append(0)
        
        # Create bar chart
        bars = plt.bar(size_labels, means, yerr=stds, capsize=10, alpha=0.7)
        
        # Add count labels
        for bar, count in zip(bars, counts):
            height = bar.get_height()
            plt.text(
                bar.get_x() + bar.get_width()/2.,
                height + 0.02,
                f'n={count}',
                ha='center', 
                va='bottom'
            )
        
        plt.title(title)
        plt.ylabel(title)
        plt.xlabel('Nodule Size')
        plt.grid(True, alpha=0.3)
        
        # Adjust y-axis for Hausdorff distance (lower is better)
        if metric == 'hausdorff':
            plt.gca().invert_yaxis()
    
    plt.tight_layout()
    plt.savefig(os.path.join(analysis_dir, 'performance_by_nodule_size.png'), dpi=300)
    plt.close()
    
    # Save detailed results to CSV
    results_data = []
    for label in size_labels:
        for metric in metrics_to_plot:
            values = results_by_size[label][metric]
            if values:
                results_data.append({
                    'Size Category': label,
                    'Metric': metric,
                    'Mean': np.mean(values),
                    'Std': np.std(values),
                    'Count': len(values)
                })
    
    results_df = pd.DataFrame(results_data)
    results_df.to_csv(os.path.join(analysis_dir, 'nodule_size_analysis.csv'), index=False)
    
    return results_by_size

def main():
    # Set random seed for reproducibility
    torch.manual_seed(42)
    np.random.seed(42)
    
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Create output directories
    output_dir = "/kaggle/working/"
    os.makedirs(output_dir, exist_ok=True)
    
    # Dataset paths
    base_dir = "/kaggle/input/luna16-npy-preprocessed/data"  # Update with your path
    
    # Create datasets
    train_dataset = Luna16Dataset2D(base_dir, split='train', target_size=(224, 224), debug=True)
    val_dataset = Luna16Dataset2D(base_dir, split='val', target_size=(224, 224), debug=True)
    test_dataset = Luna16Dataset2D(base_dir, split='test', target_size=(224, 224), debug=True)
    
    # Create dataloaders
    batch_size = 8
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    
    # Initialize TransUNet model with novel modifications
    model = TransUNet(
        img_size=224,
        in_channels=1,
        num_classes=2,
        patch_size=16,
        embed_dim=768,
        depth=12,
        num_heads=12,
        use_dual_attn=True,  # Enable dual-path attention
        use_refinement=True,  # Enable feature refinement modules
        use_multiscale_fusion=True  # Enable multi-scale skip fusion
    ).to(device)
    
    # Print model summary
    print(f"Model initialized with {sum(p.numel() for p in model.parameters())} parameters")

        # ADD THE MODEL VISUALIZATION CODE HERE
    print("\n========== Generating Model Architecture Visualization ==========")
    visualize_model_architecture(model, "model_viz")
    from torchsummary import summary
    summary(model, (1, 224, 224))

    

    
    
    # Train the model
    print("\n========== Training the Model ==========")
    model, history = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        num_epochs=50,
        device=device,
        output_dir=output_dir
    )
    
    # Load best model for evaluation
    model.load_state_dict(torch.load(os.path.join(output_dir, "best_model.pth")))
    
    # Comprehensive evaluation on test set
    print("\n========== Evaluating on Test Set ==========")
    test_results_dir = os.path.join(output_dir, "test_evaluation")
    test_metrics = evaluate_model(model, test_loader, device, output_dir=test_results_dir)
    
    # Additional analysis
    print("\n========== Generating Additional Visualizations ==========")
    
    # Analyze performance by nodule size
    visualize_nodule_size_analysis(model, test_dataset, device, output_dir)
    
    # Create attention and feature map visualizations for some test samples
    vis_dir = os.path.join(output_dir, "feature_visualizations")
    os.makedirs(vis_dir, exist_ok=True)
    
    # Get a few sample images
    sample_batch = next(iter(test_loader))
    for i in range(min(3, len(sample_batch["image"]))):
        sample_image = sample_batch["image"][i:i+1].to(device)
        
        # Save attention visualization
        attn_fig = visualize_attention_maps(model, sample_image, device)
        attn_fig.savefig(os.path.join(vis_dir, f"attention_maps_{i}.png"), dpi=300)
        plt.close(attn_fig)
        
        # Save feature map visualization
        visualize_feature_maps(model, sample_image, device, vis_dir, i)
    
    # Final visualizations of model predictions
    print("\n========== Generating Result Visualizations ==========")
    visualize_results(model, test_dataset, num_samples=10, device=device)
    
    print(f"\nAll results saved to '{output_dir}' directory.")
    print("Training and evaluation completed successfully!")

if __name__ == "__main__":
    main()



Using device: cuda
Found 1446 image files and 1446 label files
Found 1446 valid train samples with matching labels
First few matched pairs:
  images_0.npy -> masks_0.npy
  images_1.npy -> masks_1.npy
  images_10.npy -> masks_10.npy
Found 162 image files and 162 label files
Found 162 valid val samples with matching labels
First few matched pairs:
  images_0.npy -> masks_0.npy
  images_1.npy -> masks_1.npy
  images_10.npy -> masks_10.npy
Found 162 image files and 162 label files
Found 162 valid test samples with matching labels
First few matched pairs:
  images_0.npy -> masks_0.npy
  images_1.npy -> masks_1.npy
  images_10.npy -> masks_10.npy
Model initialized with 104755502 parameters

========== Generating Model Architecture Visualization ==========
Model architecture visualization saved to model_viz


/tmp/ipykernel_105/4015411800.py:301: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert H == self.img_size[0] and W == self.img_size[1], \
/tmp/ipykernel_105/3940435572.py:81: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  h = w = int(math.sqrt(x.size(1)))
/tmp/ipykernel_105/3940435572.py:87: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!

ONNX model exported to model_viz/model.onnx
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           3,136
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
      ResNetBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]            

Epoch 1/50 - Training:   0%|          | 0/181 [00:00<?, ?it/s]

Loss function - inputs shape: torch.Size([8, 2, 224, 224]), targets shape: torch.Size([8, 224, 224])


...

Performance report saved to /kaggle/working/test_evaluation/performance_report.html

========== Generating Additional Visualizations ==========


Analyzing nodule sizes:   2%|▏         | 3/162 [00:00<00:06, 24.77it/s]

Original shapes - Image: (512, 512), Mask: (512, 512)


Analyzing nodule sizes: 100%|██████████| 162/162 [00:06<00:00, 24.07it/s]


Original shapes - Image: (512, 512), Mask: (512, 512)

========== Generating Result Visualizations ==========
Saved 10 visualization samples to 'visualization_results' directory

All results saved to '/kaggle/working/' directory.
Training and evaluation completed successfully!
